# heartbeat.py

In [1]:
from paho.mqtt.client import Client
from threading import Thread
import time

# Heartbeat settings: MQTT publisher definitions -> semi fixed
PAYLOAD_LENGTH = 8       # [Bytes] => 64 bit
BYTEORDER = 'big'        # Big-Endian
CHANNEL = 'truckChannel' # Listener
INTERVAL = 0.025         # delay (25ms) between publishes

class Heartbeat(Thread):
    def __init__(self, client: Client, initialCounterValue: int = 0):
        Thread.__init__(self, daemon=True)
        self.__running__ = False
        self.__client__ = client
        self.__count__ = initialCounterValue
    
    def getCount(self): return self.__count__
    
    def run(self):
        self.__running__ = True
        while self.__running__:
            self.__heartbeat_routine__()
            time.sleep(INTERVAL)
            
    def __heartbeat_routine__(self):
        try:
            self.__count__ += 1
            payload = self.__count__.to_bytes(PAYLOAD_LENGTH, BYTEORDER)
            self.__client__.publish(topic=CHANNEL, payload=payload)
        except Exception as e:
            print(e)
            self.__running__ = False


# publisher.py

In [ ]:
# from paho.mqtt.client import Client
# from .heartbeat.py import Heartbeat

# Network Identification: MQTT Broker (self) -> fixed
ID = 'heartbeatPublisher'
ADDRESS = '127.0.0.1'
PORT = 1883
USERNAME = 'testUser'
PASSWORD = 'test'

def initialize():
    client = Client(client_id=ID, clean_session=False)
    client.username_pw_set(USERNAME, PASSWORD)
    client.connect(ADDRESS, PORT)
    return client

if __name__ == '__main__':
    startCountingAt = 0
    mqttClient = initialize()
    broker = Heartbeat(mqttClient, startCountingAt)
    broker.start()
    print('alive 1')
    broker.join()
    print('2')
    mqttClient._sock_close()
    

alive 1


# Im Anschluss

#### Ich wuerde empfehlen dann im Anschluss einen Subscriber in den Admin-Microservice herein zu packen, womit man dann den Broker monitoren kann -> dann haben wir alles zentral, auch wenn die properties dann 3 Mal hinterlegt sind

In [ ]:
# https://www.youtube.com/watch?v=72u6gIkeqUc

# net start mosquitto
# net stop mosquitto

# Nach Abschluss des Projektes die Firewall-Settings zuruecksetzen